In [31]:
%pylab
from px import *
import numpy as np

class log(object):
    def __init__(self,logname):
        
        with open(logname, 'r') as f:
             self.lines = f.readlines()
                
        # every index corresponds to a line of the log file. The value at an index
        # is a string classifying the line as a command, response, spurious interrupt,
        # or invalid sequence response
        self.all = []
        
        # lists containing the line numbers at which various types of entries occur; essentially
        # this is just a re-indexing of self.all
        self.invalid = []
        self.rsp = [] # self.rsp and cmd will be (line number,system time, command) tuples so
        self.cmd = [] # (cont) that self.cmd[n][2] has all the message information for the nth command
        self.other = []
        self.spurious = []
        
        lno = 1
        for line in self.lines:
            self.parse_line(line,lno)
            lno += 1
            
        self.analyze()
            
    def parse_line(self,line,lno):
        """
        Classify each line of the log and update the lists
        defined in __init__
        """
        
        if "sending" in line:
            self.cmd.append(self.parse_cmd_rsp(line,lno))
            self.all.append("cmd")
        elif "receiving" in line:
            self.rsp.append(self.parse_cmd_rsp(line,lno))
            self.all.append("rsp")
        elif "invalid" in line:
            self.invalid.append(lno)
            self.all.append("invalid")
        elif "Spurious" in line:
            self.spurious.append(lno)
            self.all.append("spurious")
        else:
            self.other.append(lno)
            self.all.append("other")
        
    def parse_cmd_rsp(self,line,lno):
        """
        Parse a "sending" or "receiving" line, returning
        the line number, system time, and a px.command object
        encoding the message sent to the ACB or received from
        it
        """
        
        # handle the non px fields
        l = line.split(' ')
        t = float(l[1][:-3])
        
        # format the px field for parsing 
        val = ''
        for v in l[2:]:
            if len(v) != 2:
                v = '0'+v
            val += v
                
        return lno,t,command(val)
        
    def analyze(self):
        
        
        # create arrays of the header index values for commands and responses
        self.cmd_header = np.array([c[2].header for c in self.cmd])
        self.rsp_header = np.array([c[2].header for c in self.rsp])
        
        # create arrays of the system times of each command or response
        self.cmd_time = np.array([c[1] for c in self.cmd])
        self.rsp_time = np.array([c[1] for c in self.rsp])
        
        # create arrays containing the line numbers of each command or response
        self.cmd_lno = np.array([c[0] for c in self.cmd])
        self.rsp_lno = np.array([c[0] for c in self.rsp])
        
        _all = np.array(self.all)
        
        
        # arrays that are true at an index if index+1 is a line number with the coressponding error message
        self.spurious_flag = (_all == "spurious")
        self.invalid_flag = (_all == "invalid")
        
        self.spurious_rate = np.sum(self.spurious_flag) / float(len(_all))
        self.invalid_rate = np.sum(self.invalid_flag) / float(len(_all))
        
        # approximation to the resend rate
        l = float(len(self.cmd_lno)+len(self.rsp_lno))
        
        self.cmd_resend_rate = (np.sum( np.diff(self.cmd_lno) < 2 ) + np.sum( np.diff(self.rsp_lno) > 2 )) / l
        self.rsp_resend_rate = (np.sum( np.diff(self.cmd_lno) > 2 ) + np.sum( np.diff(self.rsp_lno) < 2 )) / l

Using matplotlib backend: Qt4Agg
Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


In [26]:
log95 = log("log_RP95Hz.txt")
log80 = log("log_RP80Hz.txt")
log59 = log("log_RP59Hz.txt")

In [58]:
ylabel("Command Header Value")
xlabel("Command Number")

plot( log95.cmd_header, label = "95 Hz",linewidth=2 )
plot( log80.cmd_header, label = "80 Hz",linewidth=2 )
plot( log59.cmd_header, label = "59 Hz",linewidth=2 )

legend()

In [64]:
ylabel("Response Header Value")
xlabel("Response Number")

plot( log95.rsp_header, label = "95 Hz",linewidth=2 )
plot( log80.rsp_header, label = "80 Hz",linewidth=2 )
plot( log59.rsp_header, label = "59 Hz",linewidth=2 )

legend()

In [74]:
ylabel("d cmd position / d line number")
xlabel("Line Number")

s = 200
e = 700

#plot( np.diff(log95.cmd_lno)[s:e], label = "95 Hz",linewidth=2, color = 'b' )
#plot( np.diff(log80.cmd_lno)[s:e], label = "80 Hz",linewidth=2, color = 'g' )
plot( np.diff(log59.cmd_lno)[s:e], label = "59 Hz",linewidth=2, color = 'r' )

legend()

In [7]:
ylabel("d rsp position / d line number")
xlabel("Line Number")

s = 200
e = 700

plot( np.diff(log95.rsp_lno)[s:e], label = "95 Hz",linewidth=2, color = 'b' )
plot( np.diff(log80.rsp_lno)[s:e], label = "80 Hz",linewidth=2, color = 'g' )
plot( np.diff(log59.rsp_lno)[s:e], label = "59 Hz",linewidth=2, color = 'r' )

legend()